# Logging in Ruby

The [Logger](http://ruby-doc.org/stdlib-3.0.0/libdoc/logger/rdoc/Logger.html) class in the Ruby standard library, helps write log messages to a file or stream. It supports time- or size-based rolling of log files. Messages can be assigned severities, and only those messages at or above the logger's current reporting level will be logged.

When you write code, you simply assume that all the messages will be logged. At runtime, you can get a more or less verbose log by changing the log level. A production application usually has a log level of `Logger::INFO` or `Logger::WARN`. From least to most severe, the instance methods are `Logger.debug`, `Logger.info`, `Logger.warn`, `Logger.error` and `Logger.fatal`.

The `DEBUG` log level is useful for step-by-step diagnostics of a complex task. The `ERROR` level is often uses when handling exceptions: if the program can't solve a problem, it logs the exception rather than crash and expects a human administrator to deal with it. The `FATAL` level should only be used when the program cannot recover from a problem, and is about to crash or exit.

If your log is being stored in a file, you can have `Logger` rotate or replace the log file when it gets too big, or once a certain amount of time has elapsed:

In [1]:
require 'logger'

# Keep data for the current month only
logger = Logger.new('this_month.log', 'monthly')

# Keep data for today and the past 20 days
logger = Logger.new('application.log', 20, 'daily')

# Start the log over whenever the log exceeds 100 megabytes in size
logger = Logger.new('application.log', 0, 100 * 1024 * 1024)

#<Logger:0x00007fcf4f8334b8 @level=0, @progname=nil, @default_formatter=#<Logger::Formatter:0x00007fcf4f833490 @datetime_format=nil>, @formatter=nil, @logdev=#<Logger::LogDevice:0x00007fcf4f833440 @shift_period_suffix="%Y%m%d", @shift_size=104857600, @shift_age=0, @filename="application.log", @dev=#<File:application.log>, @binmode=false, @mon_data=#<Monitor:0x00007fcf4f833418>, @mon_data_owner_object_id=1440>>

The code below, uses the application's logger to print a debugging message, and (at a higher severity) as part of error-handling code.

In [2]:
require 'logger'

LOG_FILE='log_file.log'
$LOG = Logger.new(LOG_FILE, 'monthly')

def divide(numerator, denominator)
  $LOG.debug "Numerator: #{numerator}, denominator: #{denominator}"
  begin
    result = numerator / denominator
  rescue StandardError => e
    $LOG.error "Error in division!: #{e}"
    result = nil
  end
  result
end

divide(10, 2)
divide(10, 0)

File.open(LOG_FILE, 'r') do |file|
  while line = file.gets
    puts line
  end
end

# Logfile created on 2021-05-18 21:01:37 +0200 by logger.rb/v1.4.3

D, [2021-05-18T21:01:37.586099 #15435] DEBUG -- : Numerator: 10, denominator: 2

D, [2021-05-18T21:01:37.586176 #15435] DEBUG -- : Numerator: 10, denominator: 0

E, [2021-05-18T21:01:37.586244 #15435] ERROR -- : Error in division!: divided by 0



To change the log level, simply assign the appropriate constant to level:

In [3]:
$LOG.debug "Changing logger level from #{$LOG.level} to #{Logger::ERROR}"
$LOG.level = Logger::ERROR

3

Now our logger will ignore all log messages except thos with severity `ERROR` or `FATAL`.

In [4]:
divide(10, 2)
divide(10, 0)

File.open(LOG_FILE, 'r') do |file|
  while line = file.gets
    puts line
  end
end

# Logfile created on 2021-05-18 21:01:37 +0200 by logger.rb/v1.4.3

D, [2021-05-18T21:01:37.586099 #15435] DEBUG -- : Numerator: 10, denominator: 2

D, [2021-05-18T21:01:37.586176 #15435] DEBUG -- : Numerator: 10, denominator: 0

E, [2021-05-18T21:01:37.586244 #15435] ERROR -- : Error in division!: divided by 0

D, [2021-05-18T21:01:37.588567 #15435] DEBUG -- : Changing logger level from 0 to 3

E, [2021-05-18T21:01:37.594362 #15435] ERROR -- : Error in division!: divided by 0

